# Generadores e Iteradores

Programar es básicamente escribir condicionales, ciclos y asignaciones de variables. Aunque no de cualquier forma.

Los ciclos, ya sean ciclos `while` o iteraciones `for` son una de las estructuras más ubicuas en cualquier lenguaje. Los programas hacen muchas iteraciones para procesar listas, leer archivos, buscar en bases de datos y demás.

Una de las características más poderosas de Python es la capacidad de redefinir la iteración mediante las llamadas **"funciones generadoras"**. En esta sección veremos de que se trata ésto. Hacia el final vas a escribir programas que procesan datos **en tiempo real**, a medida que son generados.

Terminamos la clase con un ejercicio optativo que combina dos temas importantes: objetos y simulaciones. El ejercicio optativo propone simular en el espacio y tiempo la dinámica predador-presa utilizando para esto programación orientada a objetos.

## El Protocolo de Iteración

En el fondo, un for no es más que esto:

```python
for x in obj:
    # instrucciones

###

_iter = obj.__iter__()        # Buscar el objeto iterador
while True:
    try:
        x = _iter.__next__()  # Dame el siguiente item
    except StopIteration:     # No hay más items
        break
    # instrucciones ...
```

Por ejemplo, si quisiéramos iterar manualmente una lista:

In [7]:
lista = [1, 2, 3]
it = lista.__iter__()

it.__next__()

1

In [8]:
it.__next__()

2

In [9]:
it.__next__()

3

In [10]:
it.__next__()

StopIteration: 

### Construyendo un ITERABLE

```python
class Camion:
    def __init__(self):
        self.lotes = []
        
    def __iter__(self):
        return self.lotes.__iter__()
    
```

Se puede usar la función nativa de python `next()` como atajo al `__next__()`

## Generadores

Un generador es una función que **define un patrón de iteración**

In [1]:
def regresiva(n):
    while n>0:
        yield n
        n -= 1

In [4]:
for i in regresiva(10):
    print(i, end=' ')

10 9 8 7 6 5 4 3 2 1 

Un generador es cualquier función que usa el commando yield.

El comportamiento de los generadores es algo diferente al del resto de las funciones.

Al llamar a un generador creás un objeto generador, pero su función no se ejecuta de inmediato.

In [5]:
def regresiva(n):
    # Agreguemos este print para ver qué pasa...
    print('Cuenta regresiva desde', n)
    while n > 0:
        yield n
        n -= 1

In [6]:
x = regresiva(10)

In [7]:
x

<generator object regresiva at 0x7f7e18cbd120>

La función sólo se ejecuta ante un llamado al método `__next__()`

In [8]:
x.__next__()

Cuenta regresiva desde 10


10

In [9]:
a = next(x)
print(a)

9


**Observación:** Una función generadora implementa el mismo protocolo de bajo nivel que los `for` usan sobre listas, tuplas, diccionarios, archivos, etc. ¡Por eso es tan sencillo usar los generadores para iterar!

***

Si te encontrás con la necesidad de obtener una iteración particular, pensá en usar funciones generadoras. 

Son fáciles de escribir: simplemente hacé una función que implemente la lógica de iteración deseada y use yield para entregar valores.

Por ejemplo, probá este generador que busca un archivo y entrega las líneas que incluyen cierto substring.

In [11]:
def filematch(filename, substr):
    with open(filename, 'r') as f:
        for line in f:
            if substr in line:
                yield line

for line in open('../Data/camion.csv'):
    print(line, end='')

nombre,cajones,precio
Lima,100,32.2
Naranja,50,91.1
Caqui,150,103.44
Mandarina,200,51.23
Durazno,95,40.37
Mandarina,50,65.1
Naranja,100,70.44


In [13]:
for line in filematch('../Data/camion.csv', 'Naranja'):
    print(line, end='')

Naranja,50,91.1
Naranja,100,70.44


Esta idea es muy interesante: podés armar una función que **encapsule** todo el procesamiento de datos y después recorrerla con un ciclo for para que te entregue los datos uno a uno.

***

**Un generador puede ser una forma interesante de vigilar datos a medida que son producidos.**

Acá hay una serie de ejercicios hechos **MUY ZARPADOS**. Estaría bueno volver a leer esto después de un tiempo a ver si decantó. Son los ejercicios de 9.5 a 9.7

## Productores, Consumidores y Cañerías

Los *generadores* son una herramienta muy útil para configurar **pipelines** (cañerías). 

Este concepto requiere una breve aclaración: Un pipeline tradicional en computación consta de una serie de programas y archivos asociados que constituyen una estructura de procesamiento de datos, donde cada programa se ejecuta independientemente de los demás, pero juntos resultan en un flujo conveniente de datos a través de los archivos asociados desde un **"productor"** (una cámara, un sensor, un lector de código de barras) hasta un **"consumidor"** (un graficador, un interruptor eléctrico, un log de una página web)

### Sistemas productor-consumidor

El concepto de generadores está íntimamente asociado a problemas de tipo productor-consumidor en sus varias formas. Fijate esta estructura, que es típica de muchos programas:

```python

# Productor
def vigilar(f):
    ...
    while True:
        ...
        yield linea        # Produce/obtiene valores para "linea"
        ...

# Consumidor
for linea in vigilar(f):    # Consume líneas del `yield`
    ...

```

Los `yield` generan los datos que los `for` consumen.

### **Pipelines con generadores**

Podés usar esta característica de los generadores para construír pipelines que procesen tus datos, un concepto que es muy usado en Unix (pipes) pero en Windows se usa menos.

```python
productor → procesamiento → procesamiento → consumidor
```

Los pipelines de procesamiento de datos tienen un productor al comienzo, una cadena de etapas de procesamiento y un consumidor al final.

#### **Productor**

```python
def productor():
    ...
    yield item
    ...
```

El **productor** es en general un <u>generador</u>, aunque tambien podría ser una lista o cualquier otra secuencia iterable.

El `yield` alimenta al pipeline de datos.

#### **Consumidor**

```python

def consumidor(s):
    for item in s:
        ...
```

El **consumidor** es un ciclo `for`. Obtiene los elementos `item` y los usa para algo.

#### **Procesamiento**

```python
def procesamiento(s):
    for item in s:
        ...
        yield itemnuevo
        ...
```

Las etapas intermedias de procesamiento simultáneamente consumen y producen datos, pueden alterar el flujo de datos, eliminar o modificar datos según su función.

Todo junto:

```python
def productor():
    ...
    yield item          # yield devuelve un item que será recibido por `procesamiento`
    ...

def procesamiento(s):
    for item in s:      # item viene del `productor`
        ...
        yield newitem   # este yield devuelve un nuevo item
        ...

def consumidor(s):
    for item in s:      # item viene de `procesamiento`
        ...
```